<a href="https://colab.research.google.com/github/mdapoy/Machine-Learning-week-8-16/blob/main/Ch5_Support_Vector_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PENJELASAN AWAL

Chapter 5 dari buku "Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow" memperkenalkan Support Vector Machines (SVMs), sebuah model Machine Learning yang kuat dan serbaguna. Bab ini menjelaskan konsep inti SVMs, bagaimana menggunakannya untuk klasifikasi dan regresi, serta bagaimana mekanisme internalnya bekerja, termasuk "kernel trick" yang terkenal.

## TUJUAN

Tujuan utama bab ini adalah untuk memperkenalkan pembaca pada model SVM, menjelaskan prinsip dasarnya, serta menunjukkan implementasinya untuk tugas klasifikasi (linier dan non-linier) dan regresi. Pembaca diharapkan memahami bagaimana SVM beroperasi, metrik-metrik yang relevan, dan bagaimana mengelola hyperparameternya.

## RINGKASAN UTAMA

Chapter ini membahas topik-topik kunci terkait SVM:


### 1.Linear SVM Classification (Klasifikasi SVM Linier)

- Konsep Dasar: Ide fundamental di balik SVM adalah menemukan "jalan" (atau margin) terlebar yang mungkin antara kelas-kelas dalam data. Ini disebut large margin classification.
  - Hard Margin Classification: Ini adalah pendekatan yang ketat di mana semua instansi harus berada di luar "jalan" dan di sisi yang benar.
    - Kelemahan: Hanya berfungsi jika data dapat dipisahkan secara linier dan sangat sensitif terhadap outlier.
  - Soft Margin Classification: Pendekatan yang lebih fleksibel, mencari keseimbangan antara menjaga "jalan" selebar mungkin dan membatasi pelanggaran margin (instansi yang berada di dalam "jalan" atau di sisi yang salah).
    - Hyperparameter C: Mengontrol trade-off ini. Nilai C yang rendah menghasilkan margin yang lebih lebar tetapi lebih banyak pelanggaran margin, sementara nilai C yang tinggi menghasilkan lebih sedikit pelanggaran margin tetapi margin yang lebih sempit. Jika model overfitting, C dapat dikurangi untuk regularisasi.
- Sensitivitas terhadap Skala Fitur: SVM sangat sensitif terhadap skala fitur. Penting untuk menskalakan data (misalnya, menggunakan StandardScaler dari Scikit-Learn) sebelum melatih SVM.
- Implementasi dengan Scikit-Learn:
  - LinearSVC: Kelas yang direkomendasikan untuk klasifikasi SVM linier. Ia mengimplementasikan algoritma yang dioptimalkan untuk SVM linier.
  - SVC(kernel="linear"): Alternatif untuk LinearSVC, tetapi umumnya lebih lambat untuk dataset linier besar.
  - SGDClassifier(loss="hinge"): Dapat digunakan untuk melatih pengklasifikasi SVM linier menggunakan Stochastic Gradient Descent, berguna untuk tugas klasifikasi online atau dataset besar yang tidak muat di memori.
  - SVM tidak mengeluarkan probabilitas kelas secara langsung, hanya skor keputusan.


In [ ]:
# Import libraries
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import mean_squared_error

# Load Iris dataset (from Chapter 4)
iris = datasets.load_iris()
X_iris = iris["data"][:, (2, 3)]  # petal length, petal width
y_iris = (iris["target"] == 2).astype(np.float64) # Iris virginica

# 1. Linear SVM Classification
# Soft Margin Classification with LinearSVC
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42)) # C=1, loss="hinge"
])
svm_clf.fit(X_iris, y_iris)

print("Prediction for [5.5, 1.7] (Iris virginica):", svm_clf.predict([[5.5, 1.7]]))


Prediction for [5.5, 1.7] (Iris virginica): [1.]


In [ ]:
# You can try SVC with linear kernel as an alternative
svc_linear_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svc_linear", SVC(kernel="linear", C=1, random_state=42))
])
svc_linear_clf.fit(X_iris, y_iris)
print("SVC(kernel='linear') prediction for [5.5, 1.7]:", svc_linear_clf.predict([[5.5, 1.7]]))

# You can also try SGDClassifier for linear SVM
from sklearn.linear_model import SGDClassifier # Import SGDClassifier
sgd_linear_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("sgd_clf", SGDClassifier(loss="hinge", alpha=1/(len(X_iris)*1), random_state=42)) # alpha corresponds to 1/(m*C)
])
sgd_linear_clf.fit(X_iris, y_iris)
print("SGDClassifier prediction for [5.5, 1.7]:", sgd_linear_clf.predict([[5.5, 1.7]]))

SVC(kernel='linear') prediction for [5.5, 1.7]: [1.]
SGDClassifier prediction for [5.5, 1.7]: [1.]


### 2.Nonlinear SVM Classification (Klasifikasi SVM Non-Linier)

- Menangani Data Non-Linier:
  - Menambahkan Fitur Polinomial: Salah satu pendekatan adalah menambahkan fitur-fitur polinomial (seperti yang dilakukan di Chapter 4) untuk mengubah dataset agar dapat dipisahkan secara linier di ruang fitur yang lebih tinggi. PolynomialFeatures dari Scikit-Learn dapat digunakan untuk ini.
  - Kernel Trick: Metode matematis "kernel trick" memungkinkan SVM untuk mencapai hasil yang sama seolah-olah fitur polinomial telah ditambahkan, tanpa benar-benar menambahkannya. Ini menghindari ledakan kombinatorial jumlah fitur.
    - SVC Class: Kelas SVC dari Scikit-Learn mengimplementasikan kernel trick.
    - Polynomial Kernel: Digunakan untuk data polinomial. Hyperparameter degree mengontrol derajat polinomial, dan coef0 mengontrol seberapa besar model dipengaruhi oleh polinomial derajat tinggi versus derajat rendah.
- Similarity Features (Fitur Kemiripan):
  - Pendekatan lain adalah menambahkan fitur yang dihitung menggunakan fungsi kemiripan (misalnya, Gaussian Radial Basis Function - RBF) yang mengukur seberapa mirip setiap instansi dengan landmark tertentu.
  - Gaussian RBF Kernel: Kelas SVC juga mendukung kernel RBF. Hyperparameter gamma (γ) mengontrol bentuk kurva RBF. Nilai γ yang besar membuat kurva lebih sempit (rentang pengaruh instansi lebih kecil), menghasilkan batas keputusan yang lebih tidak teratur dan cenderung overfitting. Nilai γ yang kecil menghasilkan kurva yang lebih lebar (rentang pengaruh instansi lebih besar), batas keputusan yang lebih halus, dan cenderung underfitting. Jadi, γ bertindak seperti hyperparameter regularisasi.
- Memilih Kernel:
  - Sebagai aturan praktis, selalu coba linear kernel terlebih dahulu (LinearSVC lebih cepat daripada SVC(kernel="linear")).
  - Jika training set tidak terlalu besar, coba juga Gaussian RBF kernel karena ia bekerja dengan baik di sebagian besar kasus.
  - Eksperimen dengan kernel lain menggunakan cross-validation dan grid search jika waktu dan daya komputasi memungkinkan.



In [ ]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC

# Generate moons dataset (from Chapter 4)
X_moons, y_moons = make_moons(n_samples=100, noise=0.15, random_state=42)

# 2. Nonlinear SVM Classification
# Polynomial Features with LinearSVC
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=10, loss="hinge", random_state=42))
])
polynomial_svm_clf.fit(X_moons, y_moons)

print("\nPolynomial Features with LinearSVC prediction for [[-1, 1]] (example):", polynomial_svm_clf.predict([[-1, 1]]))

# Polynomial Kernel with SVC
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5, random_state=42))
])
poly_kernel_svm_clf.fit(X_moons, y_moons)

print("Polynomial Kernel SVC prediction for [[-1, 1]] (example):", poly_kernel_svm_clf.predict([[-1, 1]]))

# Gaussian RBF Kernel with SVC
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001, random_state=42)) # Example: high gamma, low C
])
rbf_kernel_svm_clf.fit(X_moons, y_moons)

print("Gaussian RBF Kernel SVC prediction for [[-1, 1]] (example):", rbf_kernel_svm_clf.predict([[-1, 1]]))


Polynomial Features with LinearSVC prediction for [[-1, 1]] (example): [0]
Polynomial Kernel SVC prediction for [[-1, 1]] (example): [0]
Gaussian RBF Kernel SVC prediction for [[-1, 1]] (example): [0]


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### 3.Computational Complexity (Kompleksitas Komputasi)

- LinearSVC: Berbasis pustaka liblinear, kompleksitas pelatihan sekitar O(m×n). Skala hampir linier dengan jumlah instansi pelatihan dan jumlah fitur. Lebih lambat jika memerlukan presisi sangat tinggi (dikontrol oleh hyperparameter tol / ϵ).
- SVC: Berbasis pustaka libsvm, mendukung kernel trick. Kompleksitas pelatihan biasanya antara O(m
2
 ×n) dan O(m
3
 ×n). Sangat lambat untuk jumlah instansi pelatihan yang besar (ratusan ribu). Cocok untuk training set yang kompleks, berukuran kecil hingga sedang. Skala baik dengan jumlah fitur, terutama dengan fitur sparse.

Tabel berikut merangkum karakteristik utama dari beberapa kelas Support Vector Machine (SVM) yang umum digunakan dalam Scikit-Learn untuk tugas klasifikasi. Pemilihan kelas yang tepat bergantung pada ukuran dataset, jumlah fitur, dan apakah masalahnya bersifat linier atau non-linier.

| Class          | Time complexity                 | Out-of-core support | Scaling required | Kernel trick |
| :------------- | :------------------------------ | :------------------ | :--------------- | :----------- |
| `LinearSVC`    | $O(m \times n)$                 | No                  | Yes              | No           |
| `SGDClassifier` | $O(m \times n)$                 | Yes                 | Yes              | No           |
| `SVC`          | $O(m^2 \times n)$ to $O(m^3 \times n)$ | No                  | Yes              | Yes          |

**Penjelasan Kolom:**

* **Class**: Nama kelas pengklasifikasi SVM di Scikit-Learn.
    * [cite_start]`LinearSVC`: Menerapkan pengklasifikasi Support Vector Classification linier.
    * [cite_start]`SGDClassifier`: Menerapkan pengklasifikasi linier menggunakan Stochastic Gradient Descent (SGD).
    * [cite_start]`SVC`: Menerapkan pengklasifikasi Support Vector Classification yang mendukung kernel non-linier.

* **Time complexity**: Menunjukkan bagaimana waktu pelatihan meningkat seiring dengan jumlah instansi ($m$) dan jumlah fitur ($n$).
    * [cite_start]$O(m \times n)$: Kompleksitas komputasi yang linier terhadap jumlah instansi dan jumlah fitur, membuatnya efisien untuk dataset besar dengan banyak fitur.
    * [cite_start]$O(m^2 \times n)$ to $O(m^3 \times n)$: Kompleksitas komputasi yang meningkat secara kuadratik atau kubik terhadap jumlah instansi, menjadikannya sangat lambat untuk dataset yang sangat besar.

* **Out-of-core support**: Menunjukkan apakah algoritma dapat menangani dataset yang tidak muat seluruhnya di memori (yaitu, dapat belajar secara inkremental dari *mini-batch* data).
    * [cite_start]`Yes`: Algoritma dapat memproses data secara inkremental.
    * [cite_start]`No`: Algoritma memerlukan seluruh dataset untuk dimuat ke dalam memori.

* **Scaling required**: Menunjukkan apakah penskalaan fitur (misalnya, Standardisasi atau Normalisasi) direkomendasikan atau diperlukan untuk kinerja optimal.
    * [cite_start]`Yes`: Penskalaan fitur sangat disarankan atau diperlukan karena algoritma sensitif terhadap skala atribut.

* **Kernel trick**: Menunjukkan apakah kelas mendukung penggunaan *kernel trick* untuk menangani masalah klasifikasi non-linier dengan memetakan data secara implisit ke ruang berdimensi lebih tinggi.
    * [cite_start]`Yes`: Kelas dapat menggunakan berbagai fungsi kernel non-linier.
    * [cite_start]`No`: Kelas hanya mendukung batas keputusan linier di ruang fitur asli.

**Catatan Tambahan:**

* [cite_start]**`LinearSVC`** adalah pilihan yang baik untuk klasifikasi linier skala besar.
* [cite_start]**`SGDClassifier`** dengan `loss="hinge"` setara dengan SVM linier dan mendukung pembelajaran *out-of-core*, sehingga cocok untuk dataset yang sangat besar.
* [cite_start]**`SVC`** adalah pilihan serbaguna untuk masalah non-linier berukuran kecil hingga menengah karena kemampuannya menggunakan *kernel trick*.



### 4.SVM Regression (Regresi SVM)

- Konsep: Membalikkan tujuan klasifikasi. Regresi SVM mencoba memasukkan sebanyak mungkin instansi ke dalam "jalan" sambil membatasi pelanggaran margin (instansi di luar "jalan"). Lebar "jalan" dikontrol oleh hyperparameter ϵ.
- ϵ-Insensitive: Menambahkan lebih banyak instansi pelatihan di dalam margin tidak memengaruhi prediksi model, sehingga model disebut ϵ-insensitive.
- Implementasi dengan Scikit-Learn:
  - LinearSVR: Untuk Regresi SVM linier, memiliki kompleksitas O(m×n) dan skala linier dengan ukuran training set.
  - SVR: Untuk Regresi SVM non-linier (mendukung kernel trick), tetapi sangat lambat untuk training set yang besar.


In [ ]:
# 4. SVM Regression
# Generate some linear data (from Chapter 4)
X_reg_svm = 2 * np.random.rand(100, 1)
y_reg_svm = 4 + 3 * X_reg_svm + np.random.randn(100, 1)

# Scale the data first for SVR (important for most regularized models)
scaler_reg_svm = StandardScaler()
X_reg_svm_scaled = scaler_reg_svm.fit_transform(X_reg_svm)
y_reg_svm_scaled = scaler_reg_svm.fit_transform(y_reg_svm) # Scale target as well if needed, but not strictly required for Huber loss

from sklearn.svm import LinearSVR, SVR

# Linear SVR
svm_reg = LinearSVR(epsilon=1.5, random_state=42) # epsilon controls margin width
svm_reg.fit(X_reg_svm_scaled, y_reg_svm_scaled.ravel()) # ravel() to flatten y
print("\nLinear SVR prediction for scaled [[1.5]]:", svm_reg.predict([[1.5]]))

# Non-linear SVR with polynomial kernel
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X_reg_svm_scaled, y_reg_svm_scaled.ravel())
print("Polynomial SVR prediction for scaled [[1.5]]:", svm_poly_reg.predict([[1.5]]))


Linear SVR prediction for scaled [[1.5]]: [1.34053499]
Polynomial SVR prediction for scaled [[1.5]]: [0.29404287]


### 5.Under the Hood (Di Balik Layar)

Bagian ini menjelaskan detail matematis cara kerja SVM, yang dapat dilewati oleh pembaca yang baru memulai ML.

* **Fungsi Keputusan dan Prediksi**: Pengklasifikasi SVM linier memprediksi kelas instansi baru $\mathbf{x}$ dengan menghitung fungsi keputusan $\mathbf{w}^T \mathbf{x} + b$. [cite_start]Jika hasilnya positif, kelas prediksi adalah kelas positif (1), jika tidak, kelas negatif (0).
    * [cite_start]$\mathbf{w}$: vektor bobot fitur.
    * [cite_start]$b$: *bias term*.
* [cite_start]**Batas keputusan** adalah himpunan titik di mana fungsi keputusan sama dengan 0.
* [cite_start]**Tujuan pelatihan** adalah menemukan $\mathbf{w}$ dan $b$ yang memaksimalkan *margin* (yaitu, meminimalkan $||\mathbf{w}||$, yang setara dengan meminimalkan $\frac{1}{2}\mathbf{w}^T \mathbf{w}$) sambil menghindari atau membatasi pelanggaran *margin*.

* **Tujuan Pelatihan (Hard Margin)**:
    * Meminimalkan $\frac{1}{2}\mathbf{w}^T \mathbf{w}$ tunduk pada kendala $t^{(i)}(\mathbf{w}^T \mathbf{x}^{(i)} + b) \ge 1$ untuk semua instansi $i$. [cite_start]($t^{(i)}$ adalah -1 untuk kelas negatif dan 1 untuk kelas positif).

* **Tujuan Pelatihan (Soft Margin)**:
    * [cite_start]Meminimalkan $\frac{1}{2}\mathbf{w}^T \mathbf{w} + C \sum_{i=1}^{m} \zeta^{(i)}$ tunduk pada kendala $t^{(i)}(\mathbf{w}^T \mathbf{x}^{(i)} + b) \ge 1 - \zeta^{(i)}$ dan $\zeta^{(i)} \ge 0$ untuk semua instansi $i$.
    * [cite_start]$\zeta^{(i)}$ adalah *slack variable* yang mengukur seberapa banyak instansi ke-$i$ diizinkan melanggar *margin*.
    * [cite_start]$C$ adalah *hyperparameter* yang mengatur *trade-off* antara tujuan *margin* besar dan pelanggaran *margin* kecil.

* [cite_start]**Quadratic Programming (QP)**: Masalah *hard margin* dan *soft margin* adalah masalah optimasi kuadratik *convex* dengan kendala linier, yang dikenal sebagai masalah *Quadratic Programming* (QP).

* **The Dual Problem (Masalah Dual)**:
    * Setiap masalah optimasi yang dibatasi (masalah *primal*) memiliki masalah *dual* yang terkait erat. [cite_start]Untuk SVM, masalah *dual* memiliki solusi yang sama dengan masalah *primal*.
    * [cite_start]Masalah *dual* seringkali lebih cepat dipecahkan ketika jumlah instansi pelatihan lebih kecil dari jumlah fitur.
    * [cite_start]Yang lebih penting, masalah *dual* memungkinkan *kernel trick*.

* **Kernelized SVMs**:
    * [cite_start]*Kernel trick* memungkinkan untuk menghitung *dot product* $\phi(\mathbf{a})^T \phi(\mathbf{b})$ di ruang fitur berdimensi tinggi (mungkin tak terbatas) hanya berdasarkan vektor asli $\mathbf{a}$ dan $\mathbf{b}$, tanpa perlu secara eksplisit mengubah vektor tersebut ke ruang fitur yang lebih tinggi.
    * [cite_start]Contoh *kernel* umum: Linear, Polynomial, Gaussian RBF, Sigmoid.
    * [cite_start]**Teorema Mercer**: Menjamin bahwa jika fungsi $K(\mathbf{a}, \mathbf{b})$ memenuhi kondisi tertentu, maka ada fungsi pemetaan $\phi$ sehingga $K(\mathbf{a}, \mathbf{b}) = \phi(\mathbf{a})^T \phi(\mathbf{b})$.
    * [cite_start]Prediksi SVM *kernelized* hanya melibatkan *dot product* antara vektor input baru dengan *support vectors*, bukan semua instansi pelatihan.

* **Online SVMs**:
    * [cite_start]Untuk SVM linier, *online learning* dapat diimplementasikan menggunakan *Gradient Descent* untuk meminimalkan fungsi biaya yang berasal dari masalah *primal*.
    * [cite_start]**Hinge Loss**: Fungsi kerugian umum yang digunakan dalam SVM.

### 6.Online SVMs (SVM Online)

- Untuk SVM linier, online learning dapat diimplementasikan menggunakan Gradient Descent untuk meminimalkan fungsi biaya yang berasal dari masalah primal.
- Hinge Loss: Fungsi kerugian umum yang digunakan dalam SVM adalah hinge loss function, didefinisikan sebagai max(0,1−t). Ini adalah 0 ketika t≥1, dan derivatifnya adalah -1 jika t<1 dan 0 jika t>1. Ini tidak dapat didiferensiasi pada t=1.

### Konsep Kunci yang Dipelajari di "Under the Hood"

- Fungsi Keputusan: Peran w dan b dalam menentukan batas klasifikasi.
- Tujuan Optimasi (Primal): Memaksimalkan margin (meminimalkan norma bobot) dengan atau tanpa slack variables.
- Pemrograman Kuadratik (QP): Klasifikasi SVM sebagai masalah optimasi QP.
- Masalah Dual: Alternatif formulasi masalah optimasi yang memungkinkan "kernel trick" dan seringkali lebih efisien.
- Kernel Trick: Transformasi implisit ke ruang berdimensi tinggi untuk menangani non-linieritas tanpa komputasi eksplisit.
- Hinge Loss: Fungsi kerugian spesifik untuk SVM.
- Support Vectors: Instansi pelatihan yang memengaruhi model, dengan Lagrange multipliers α i >0.


## KESIMPULAN

Bagian "Under the Hood" ini memberikan wawasan mendalam tentang dasar matematis yang kompleks namun elegan di balik SVM. Dengan memahami masalah optimasi primal dan dual, serta bagaimana "kernel trick" bekerja, pembaca memperoleh pemahaman yang lebih kuat tentang mengapa SVM sangat efektif untuk berbagai masalah klasifikasi dan regresi. Meskipun detail matematisnya mungkin menantang, pemahaman konsep-konsep ini sangat berharga untuk diagnosis masalah dan penyetelan model tingkat lanjut.
